# 제품 이상 여부 판별 프로젝트

# 1. 데이터 & 라이브러리 불러오기

In [1]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings; warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

### 데이터 읽어오기

In [3]:
ROOT_DIR = "data"
RANDOM_STATE = 77

# Load data
train_data = pd.read_csv(os.path.join("train.csv"))
test_data = pd.read_csv(os.path.join("test.csv"))
print("train shape : ", train_data.shape)
print("test shape : ", test_data.shape)

train shape :  (40506, 464)
test shape :  (17361, 465)


# 2. 데이터 전처리 - 1

### Null값 확인

In [4]:
train_data.isna().sum()

Wip Line_Dam                         0
Process Desc._Dam                    0
Equipment_Dam                        0
Model.Suffix_Dam                     0
Workorder_Dam                        0
                                 ...  
Receip No Judge Value_Fill2      40506
WorkMode Collect Result_Fill2        0
WorkMode Unit Time_Fill2         40506
WorkMode Judge Value_Fill2       40506
target                               0
Length: 464, dtype: int64

### Null이 50%이상인 컬럼제거


In [5]:
# Null 값이 포함된 컬럼만 추출
null_columns = train_data.columns[train_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = train_data[null_columns].isna().sum()

# Drop columns with more than half of the values missing
drop_cols = []
for column in train_data.columns:
    if (train_data[column].notnull().sum() // 2) < train_data[
        column
    ].isnull().sum():
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)
print("train shape : ", train_data.shape)

train shape :  (40506, 181)


In [6]:
features = []
for col in train_data.columns:
    try:
        features.append(col)
    except:
        continue

In [7]:
test_data = test_data[features]
print("test shape : ", test_data.shape)

test shape :  (17361, 181)


# 3. 데이터 전처리 - 2

### 다시 NULL값 확인


In [8]:
# Null 값이 포함된 컬럼만 추출
null_columns = train_data.columns[train_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = train_data[null_columns].isna().sum()

null_counts

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam      12766
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1    12766
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2    12766
dtype: int64

In [9]:
# 컬럼 이름 설정
columns_of_interest = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 컬럼의 유니크 값 찾기
for column in columns_of_interest:
    unique_values = train_data[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print()  # 빈 줄 추가

Unique values in column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam':
[nan '550.3' 'OK' '162.4' '549' '549.5' '550' '548.5']

Unique values in column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1':
[nan '838.4' 'OK' '837.7' '837.9' '838.2' '837.5']

Unique values in column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2':
[nan '835.5' 'OK' '305']



NULL, 숫자형, 범주형 변수가 섞여있음 -> 데이터가 밀린것을 확인

### 데이터 밀려있는부분 전처리

#### Train - Dam 밀린부분 처리

In [11]:
Dam1[Dam_col]

,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position Y Collect Result_Dam,HEAD Standby Position Z Collect Result_Dam,Head Clean Position X Collect Result_Dam,Head Clean Position Y Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position X Collect Result_Dam,Head Purge Position Y Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position X Collect Result_Dam,Head Zero Position Y Collect Result_Dam,Head Zero Position Z Collect Result_Dam,Machine Tact time Collect Result_Dam,PalletID Collect Result_Dam,Production Qty Collect Result_Dam,Receip No Collect Result_Dam,Stage1 Circle1 Distance Speed Collect Result_Dam,Stage1 Circle2 Distance Speed Collect Result_Dam,Stage1 Circle3 Distance Speed Collect Result_Dam,Stage1 Circle4 Distance Speed Collect Result_Dam,Stage1 Line1 Distance Speed Collect Result_Dam,Stage1 Line2 Distance Speed Collect Result_Dam,Stage1 Line3 Distance Speed Collect Result_Dam,Stage1 Line4 Distance Speed Collect Result_Dam,Stage2 Circle1 Distance Speed Collect Result_Dam,Stage2 Circle2 Distance Speed Collect Result_Dam,Stage2 Circle3 Distance Speed Collect Result_Dam,Stage2 Circle4 Distance Speed Collect Result_Dam,Stage2 Line1 Distance Speed Collect Result_Dam,Stage2 Line2 Distance Speed Collect Result_Dam,Stage2 Line3 Distance Speed Collect Result_Dam,Stage2 Line4 Distance Speed Collect Result_Dam,Stage3 Circle1 Distance Speed Collect Result_Dam,Stage3 Circle2 Distance Speed Collect Result_Dam,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,WorkMode Collect Result_Dam
0,NaN,550.5,464.3,161.4,384.0,383.1,384.0,274.510,274.510,274.510,257,66,0.0,127.5,66.0,124.00,257,66.0,130.85,505.0,300.0,265.00,58.5,7,127,1,5800,5800,5800,5800,5800,5800,5600,5800,5300,5300,5300,5300,5300,5300,5300,5300,5800,5800,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000
2,OK,162.4,465.4,551.7,1271.8,1383.9,1271.8,274.330,274.330,274.330,257,66,0.0,127.5,66.0,124.00,257,66.0,130.85,505.0,300.0,265.02,76.9,10,73,1,5800,5800,5800,5800,5800,5800,5800,5800,5300,5300,5300,5300,5300,5300,5300,5300,5800,5800,5800,5800,5800,5800,5800,5800,0.012,-0.022,0.003
6,OK,550.4,463.7,161.3,377.6,377.1,377.6,282.500,282.500,282.500,257,66,0.0,127.5,66.0,130.85,257,66.0,130.85,505.0,300.0,265.02,53.5,8,483,1,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,0.000,0.000,0.000
7,OK,162.7,465.7,552.0,1271.8,1383.9,1271.8,280.894,280.894,280.894,257,66,0.0,127.5,66.0,130.85,257,66.0,130.85,505.0,300.0,265.02,57.8,11,105,1,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,5500,6500,5500,6500,6500,6500,6500,6500,6500,6500,6500,6500,0.000,0.000,0.000
10,OK,162.4,465.4,551.7,1271.8,1383.9,1271.8,274.330,274.330,274.330,257,66,0.0,127.5,66.0,124.00,257,66.0,130.85,505.0,300.0,265.02,76.8,15,78,1,5800,5800,5800,5800,5800,5800,5800,5800,5300,5300,5300,5300,5300,5300,5300,5300,5800,5800,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [13]:
# Dam 안밀린부분 떼어내기
D0 = train_data.iloc[:, :24]

# Dam 밀린부분 작업해주기
Dam1 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] == "OK") |
                  (pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]
Dam2 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] != "OK") &
                  (~pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]

Dam_col = train_data.columns[24:77]

D1 = Dam1[Dam_col]
D1 = D1.iloc[:, 1:]
D1
D1['new'] = 7
D1.columns = Dam_col
D2 = Dam2[Dam_col]
D = pd.concat([D1, D2])
D

,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position Y Collect Result_Dam,HEAD Standby Position Z Collect Result_Dam,Head Clean Position X Collect Result_Dam,Head Clean Position Y Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position X Collect Result_Dam,Head Purge Position Y Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position X Collect Result_Dam,Head Zero Position Y Collect Result_Dam,Head Zero Position Z Collect Result_Dam,Machine Tact time Collect Result_Dam,PalletID Collect Result_Dam,Production Qty Collect Result_Dam,Receip No Collect Result_Dam,Stage1 Circle1 Distance Speed Collect Result_Dam,Stage1 Circle2 Distance Speed Collect Result_Dam,Stage1 Circle3 Distance Speed Collect Result_Dam,Stage1 Circle4 Distance Speed Collect Result_Dam,Stage1 Line1 Distance Speed Collect Result_Dam,Stage1 Line2 Distance Speed Collect Result_Dam,Stage1 Line3 Distance Speed Collect Result_Dam,Stage1 Line4 Distance Speed Collect Result_Dam,Stage2 Circle1 Distance Speed Collect Result_Dam,Stage2 Circle2 Distance Speed Collect Result_Dam,Stage2 Circle3 Distance Speed Collect Result_Dam,Stage2 Circle4 Distance Speed Collect Result_Dam,Stage2 Line1 Distance Speed Collect Result_Dam,Stage2 Line2 Distance Speed Collect Result_Dam,Stage2 Line3 Distance Speed Collect Result_Dam,Stage2 Line4 Distance Speed Collect Result_Dam,Stage3 Circle1 Distance Speed Collect Result_Dam,Stage3 Circle2 Distance Speed Collect Result_Dam,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,WorkMode Collect Result_Dam
0,550.5,464.3,161.4,384.0,383.1,384.0,274.510,274.510,274.510,257,66,0.0,127.5,66.0,124.00,257,66.0,130.85,505.0,300.0,265.00,58.5,7,127,1,5800,5800,5800,5800,5800,5800,5600,5800,5300,5300,5300,5300,5300,5300,5300,5300,5800,5800,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000
2,162.4,465.4,551.7,1271.8,1383.9,1271.8,274.330,274.330,274.330,257,66,0.0,127.5,66.0,124.00,257,66.0,130.85,505.0,300.0,265.02,76.9,10,73,1,5800,5800,5800,5800,5800,5800,5800,5800,5300,5300,5300,5300,5300,5300,5300,5300,5800,5800,5800,5800,5800,5800,5800,5800,0.012,-0.022,0.003
6,550.4,463.7,161.3,377.6,377.1,377.6,282.500,282.500,282.500,257,66,0.0,127.5,66.0,130.85,257,66.0,130.85,505.0,300.0,265.02,53.5,8,483,1,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,0.000,0.000,0.000
7,162.7,465.7,552.0,1271.8,1383.9,1271.8,280.894,280.894,280.894,257,66,0.0,127.5,66.0,130.85,257,66.0,130.85,505.0,300.0,265.02,57.8,11,105,1,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,6500,5500,6500,5500,6500,6500,6500,6500,6500,6500,6500,6500,6500,0.000,0.000,0.000
10,162.4,465.4,551.7,1271.8,1383.9,1271.8,274.330,274.330,274.330,257,66,0.0,127.5,66.0,124.00,257,66.0,130.85,505.0,300.0,265.02,76.8,15,78,1,5800,5800,5800,5800,5800,5800,5800,5800,5300,5300,5300,5300,5300,5300,5300,5300,5800,5800,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40495,550.6,463.9,161.5,377.6,377.1,37

#### Train - AutoClave 밀린부분 처리

In [11]:
# Auto Clave는 안밀려있음
Auto = train_data.iloc[:, 77:96]

#### Train - Fill1 밀린부분 처리

In [12]:
# Fill1 안밀린부분 작업해주기
F1_0 = train_data.iloc[:, 96:110]

# Fill1 밀린부분 작업해주기
Fill1_1 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] == "OK") |
               (pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]
Fill1_2 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] != "OK") &
               (~pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]

Fill1_col = train_data.columns[110:133]
Fill1_col
F1_1 = Fill1_1[Fill1_col]
F1_1 = F1_1.iloc[:, 1:]
F1_1['new'] = 7
F1_1.columns = Fill1_col
F1_2 = Fill1_2[Fill1_col]
FF1 = pd.concat([F1_1, F1_2])

#### Train - Fill2 밀린부분 처리

In [13]:
# Fill2 안밀린부분 작업해주기
F2_0 = train_data.iloc[:, 133:157]
F2_0
# Fill2 밀린부분 작업해주기
Fill2_1 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] == "OK") |
               (pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]
Fill2_2 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] != "OK") &
               (~pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]

Fill2_col = train_data.columns[157:180]

F2_1 = Fill2_1[Fill2_col]
F2_1 = F2_1.iloc[:, 1:]
F2_1['new'] = 0
F2_1.columns = Fill2_col
F2_2 = Fill2_2[Fill2_col]
FF2 = pd.concat([F2_1, F2_2])

### Train data 병합

In [14]:
# target
target = train_data.iloc[:, 180]

# 병합
train_data = pd.concat([D0, D, Auto, F1_0, FF1, F2_0, FF2, target], axis=1)

In [15]:
# Null 값이 포함된 컬럼만 추출
null_columns = train_data.columns[train_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = train_data[null_columns].isna().sum()

null_counts

Series([], dtype: float64)

NULL 값이 없는것을 확인

### Test data도 동일하게 처리

In [16]:
# Dam 안밀린부분 떼어내기
D0 = test_data.iloc[:, :24]

# Dam 밀린부분 작업해주기
Dam1 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] == "OK") |
                  (pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]
Dam2 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] != "OK") &
                  (~pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]

Dam_col = test_data.columns[24:77]

D1 = Dam1[Dam_col]
D1 = D1.iloc[:, 1:]
D1['new'] = 7
D1.columns = Dam_col
D2 = Dam2[Dam_col]
D = pd.concat([D1, D2])

In [17]:
# Auto Clave는 안밀려있음
Auto = test_data.iloc[:, 77:96]

In [18]:
# Fill1 안밀린부분 작업해주기
F1_0 = test_data.iloc[:, 96:110]

# Fill1 밀린부분 작업해주기
Fill1_1 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] == "OK") |
               (pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]
Fill1_2 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] != "OK") &
               (~pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]

Fill1_col = test_data.columns[110:133]
Fill1_col
F1_1 = Fill1_1[Fill1_col]
F1_1 = F1_1.iloc[:, 1:]
F1_1['new'] = 7
F1_1.columns = Fill1_col
F1_2 = Fill1_2[Fill1_col]
FF1 = pd.concat([F1_1, F1_2])

In [19]:
# Fill2 안밀린부분 작업해주기
F2_0 = test_data.iloc[:, 133:157]
F2_0
# Fill2 밀린부분 작업해주기
Fill2_1 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] == "OK") |
               (pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]
Fill2_2 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] != "OK") &
               (~pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]

Fill2_col = test_data.columns[157:180]

F2_1 = Fill2_1[Fill2_col]
F2_1 = F2_1.iloc[:, 1:]
F2_1['new'] = 0
F2_1.columns = Fill2_col
F2_2 = Fill2_2[Fill2_col]
FF2 = pd.concat([F2_1, F2_2])

In [20]:
# 병합
test_data = pd.concat([D0, D, Auto, F1_0, FF1, F2_0, FF2], axis=1)

In [21]:
# Null 값이 포함된 컬럼만 추출
null_columns = test_data.columns[test_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = test_data[null_columns].isna().sum()

null_counts

Series([], dtype: float64)

### 데이터 타입 통일

In [22]:
# 숫자형과 문자열이 섞여있는 열을 찾고 숫자형으로 변환
for column in train_data.columns:
    if train_data[column].apply(lambda x: isinstance(x, (int, float))).any() and train_data[column].apply(lambda x: isinstance(x, str)).any():
        train_data[column] = pd.to_numeric(train_data[column], errors='coerce')

In [23]:
# 숫자형과 문자열이 섞여있는 열을 찾고 숫자형으로 변환
for column in test_data.columns:
    if test_data[column].apply(lambda x: isinstance(x, (int, float))).any() and test_data[column].apply(lambda x: isinstance(x, str)).any():
        test_data[column] = pd.to_numeric(test_data[column], errors='coerce')

# 4. 데이터 전처리 - 3

In [24]:
# 불필요한 변수 제거
def variable_cleanup(data):
    # Workorder
    data_cleaned = data.drop(columns=['Workorder_Fill1', 'Workorder_AutoClave', 'Workorder_Fill2'])
    data_cleaned = data_cleaned.rename(columns={'Workorder_Dam': 'Workorder'})

    # Model.Suffix
    data_cleaned = data_cleaned.drop(columns=['Model.Suffix_Fill1', 'Model.Suffix_AutoClave', 'Model.Suffix_Fill2'])
    data_cleaned = data_cleaned.rename(columns={'Model.Suffix_Dam': 'Model.Suffix'})

    # Wip Line, Process Desc, Insp. Seq No., Insp Judge Code, WorkMode , Equipment_AutoClave
    data_cleaned = data_cleaned.drop(columns=[
        'Wip Line_Dam', 'Wip Line_Fill1', 'Wip Line_AutoClave', 'Wip Line_Fill2',
        'Process Desc._Dam', 'Process Desc._Fill1', 'Process Desc._AutoClave', 'Process Desc._Fill2',
        'Insp. Seq No._Dam', 'Insp. Seq No._Fill1', 'Insp. Seq No._AutoClave', 'Insp. Seq No._Fill2',
        'Insp Judge Code_Dam', 'Insp Judge Code_Fill1', 'Insp Judge Code_AutoClave', 'Insp Judge Code_Fill2',
        'WorkMode Collect Result_Dam', 'WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2',
        'Equipment_AutoClave'
    ])

    # unique value = 0 또는 OK (only, 좌표 조합은 제외)
    data_cleaned = data_cleaned.drop(columns=[
        'DISCHARGED SPEED OF RESIN Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2',
        'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2',
        'Dispense Volume(Stage1) Collect Result_Fill2', 'Dispense Volume(Stage2) Collect Result_Fill2', 'Dispense Volume(Stage3) Collect Result_Fill2',
        '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave'
    ])

    # 새로운 column drop
    data_cleaned = data_cleaned.drop(columns=[
        'CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam',
        'CURE START POSITION Z Collect Result_Dam',
        'HEAD Standby Position X Collect Result_Dam', 'HEAD Standby Position Y Collect Result_Dam', 'HEAD Standby Position Z Collect Result_Dam',
        'Head Clean Position X Collect Result_Dam', 'Head Clean Position Y Collect Result_Dam',
        'Head Purge Position X Collect Result_Dam', 'Head Purge Position Y Collect Result_Dam',
        'Head Zero Position X Collect Result_Dam',
        'HEAD Standby Position X Collect Result_Fill1','HEAD Standby Position Y Collect Result_Fill1','HEAD Standby Position Z Collect Result_Fill1',
        'Head Clean Position X Collect Result_Fill1','Head Clean Position Y Collect Result_Fill1','Head Clean Position Z Collect Result_Fill1',
        'Head Purge Position X Collect Result_Fill1','Head Purge Position Y Collect Result_Fill1',
        'CURE END POSITION Θ Collect Result_Fill2',
        'CURE STANDBY POSITION X Collect Result_Fill2', 'CURE STANDBY POSITION Θ Collect Result_Fill2',
        'CURE START POSITION Θ Collect Result_Fill2',
        'HEAD Standby Position X Collect Result_Fill2','HEAD Standby Position Y Collect Result_Fill2','HEAD Standby Position Z Collect Result_Fill2',
        'Head Clean Position X Collect Result_Fill2','Head Clean Position Y Collect Result_Fill2','Head Clean Position Z Collect Result_Fill2',
        'Head Purge Position X Collect Result_Fill2','Head Purge Position Y Collect Result_Fill2'
    ])

    return data_cleaned

train_data = variable_cleanup(train_data); test_data = variable_cleanup(test_data)
print(train_data.shape); print(test_data.shape)

(40506, 113)
(17361, 112)


### Tact time 제외하고 모두 범주형 처리

In [25]:
# 범주형 및 연속형 변수 정의
numerical_cols = ['Machine Tact time Collect Result_Fill2', 'Machine Tact time Collect Result_Fill1', 'Machine Tact time Collect Result_Dam']
categorical_cols = list(set(train_data.columns) - set(numerical_cols) - set(['target']))
target_col = 'target'

len(categorical_cols)

109

In [26]:
len(numerical_cols) + len(categorical_cols)

112

### Train - Validation 구분

In [27]:
# train set, validation set 구분
train_data[categorical_cols] = train_data[categorical_cols].astype('str')
test_data[categorical_cols] = test_data[categorical_cols].astype('str')
train_data['target'] = train_data['target'].apply(lambda x: 1 if x == 'AbNormal' else 0)

X = train_data.drop('target', axis=1)
y = train_data['target']

# 4. 모델링

## Voting

In [30]:
params1={'iterations': 658,
        'depth': 5,
        'learning_rate': 0.05082916242908085,
        'class_weights': {0:1, 1:8.015222449552683},
        'l2_leaf_reg': 7.510174770627344}
params2={'iterations': 999,
        'depth': 6,
        'learning_rate': 0.03822382444156112,
        'class_weights': {0:1, 1:8.637597152124888},
        'l2_leaf_reg': 3.3559037547107047}
params3={'iterations': 1021,
        'depth': 6,
        'learning_rate': 0.03815933742323573,
        'class_weights': {0:1, 1:9.427611426765473},
        'l2_leaf_reg': 3.631569570817471}
params4={'iterations': 652,
        'depth': 5,
        'learning_rate': 0.036251727686565996,
        'class_weights': {0:1, 1:8.809186925675473},
        'l2_leaf_reg': 3.681415978241256}
params5={'iterations': 791,
        'depth': 4,
        'learning_rate': 0.02698725907627657,
        'class_weights': {0:1, 1:8.220711939778287},
        'l2_leaf_reg': 5.490510494004826}
params6={'iterations': 873,
        'depth': 6,
        'learning_rate': 0.058237302882905025,
        'class_weights': {0:1, 1:8.57569158023923},
        'l2_leaf_reg': 8.197147726760026}

In [31]:
model1 = CatBoostClassifier(**params1, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150)
model2 = CatBoostClassifier(**params2, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150)
model3 = CatBoostClassifier(**params3, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150)
model4 = CatBoostClassifier(**params4, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150)
model5 = CatBoostClassifier(**params5, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150)
model6 = CatBoostClassifier(**params6, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150)

# 5. Submission

In [32]:
# VotingClassifier 정의 (Soft Voting 사용)
voting_clf = VotingClassifier(
    estimators=[('cat1', model1), ('cat2', model2), ('cat3', model3),
               ('cat4', model4), ('cat5', model5), ('cat6', model6)
               ],
    voting='soft'  # soft, hard
            )

# VotingClassifier 학습
voting_clf.fit(X, y)
# 예측 및 성능 평가
y_pred = voting_clf.predict(test_data)
pd.DataFrame(y_pred).value_counts()

0
0    16298
1     1063
Name: count, dtype: int64

In [33]:
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub["target"].value_counts()

target
Normal      16298
AbNormal     1063
Name: count, dtype: int64

In [34]:
# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)